In [59]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

In [3]:
train_data = pd.read_csv("train_data.csv")
#list(train_data.iloc[:500,1].astype(str))
train_data

,index,title,segment_title,context,segment_context,time
0,0,為什麼日本10月，自殺人數會破2000人？,/為什麼/日本/10/月/，/自殺/人數/會破/2000/人/？,\n剛看了一下日本警視廳的資料\n今年1~9月的自殺人數\n都在1400~1800人左右在跑...,\n/剛/看/了/一下/日本/警視廳/的/資料/\n/今年/1/~/9/月/的/自殺/人數/...,20:43:35
1,1,急！問一首歌，高手請進,/急/！/問/一/首/歌/，/高手/請進,\n宅宅我最近滑臉書\n\n的影片看到一個女的\n\n右手舉起來，甩奶+抖臀\n\n左手舉起...,\n/宅宅/我/最近/滑/臉書/\n/\n/的/影片/看到/一/個/女/的/\n/\n/右手...,16:10:51
2,2,話說唐吉軻德的消息呢?,/話/說/唐吉軻德/的/消息/呢/?,\n※ 八卦板務請到 GossipPicket 檢舉板實名詢問\n※ a.張貼問卦請注意，充...,\n/※/ /八卦板/務請/到/ /GossipPicket/ /檢舉/板/實名/詢問/\n...,17:21:33
3,3,網議愛莉莎莎身材如何? 鄉民:根本藏不住,/網議/愛莉/莎莎/身材/如何/?/ /鄉民/:/根本/藏不住,\n各位常常質疑我們莎莎的身材\n\n畢竟平常莎莎穿得比較保守\n\n但昨天在莎莎的影片中\...,\n/各/位/常常/質疑/我們/莎莎/的/身材/\n/\n/畢竟/平常/莎莎/穿得/比較/保...,01:27:23
4,4,改機車的人怎麼不先去減肥或看醫生啊？,/改/機車/的/人/怎麼/不/先去/減肥/或/看醫生/啊/？,\n每次在路上看到一堆機車蹦蹦叫的\n\n明明就慢到笑，但是吵到死，我認真建議你要不要先去看...,\n/每/次/在/路上/看到/一/堆/機車/蹦蹦/叫/的/\n/\n/明明/就/慢到/笑/，...,10:26:44
...,...,...,...,...,...,...
31865,31865,濟州島是那裡比澎湖好玩,/濟州島/是/那裡/比/澎湖/好玩,\n韓國人蜜月會去濟州島\n\n台灣人蜜月不會去澎湖群島\n\n\n一樣是島\n\n澎湖應該...,\n/韓國人/蜜月/會去/濟州島/\n/\n/台灣人/蜜月/不會/去/澎湖群島/\n/\n/...,13:05:25
31866,31866,中天Youtube是不是更敢講了？,/中天/Youtube/是/不/是/更/敢/講/了/？,\n以前\n中天新聞各種誇張不實\n但還是會稍微裝一下正派\n\n現在\n中天新聞轉Yout...,\n/以前/\n/中天/新聞/各/種/誇張/不實/\n/但/還是/會/稍微/裝/一下/正派/...,22:21:53
31867,31867,要不要喝水是哪一部電影,/要不要/喝水/是/哪/一部/電影,\n只記得男主角問女主角要不要喝水\n\n這是那部\n\n想複習一下\n\n說也奇怪整部電影...,\n/只/記得/男/主角/問女/主角/要不要/喝水/\n/\n/這是/那/部/\n/\n/想...,16:41:36
31868,31868,有個和尚一直哭？,/有/個/和尚/一直/哭/？,\n有個和尚眼睛白白的\n\n看起來應該是個瞎子\n\n聽說他的實力是團隊最強的\n\n整天...,\n/有/個/和尚/眼睛/白白/的/\n/\n/看起來/應該/是/個/瞎子/\n/\n/聽說...,23:53:18


In [5]:
train_label = pd.read_csv("train_label.csv")
train_label

,index,push,down
0,0,0,0
1,1,1,0
2,2,0,0
3,3,0,1
4,4,1,0
...,...,...,...
31865,31865,1,0
31866,31866,0,0
31867,31867,0,0
31868,31868,0,0


In [7]:
# 將data與label放在一起
from glob import glob

files = glob('train*.csv')

df_list = [pd.read_csv(file) for file in files]  #串列中包含兩個Pandas DataFrame
my_train_data = pd.merge(df_list[0], df_list[1], on='index')
 
my_train_data

,index,push,down,title,segment_title,context,segment_context,time
0,0,0,0,為什麼日本10月，自殺人數會破2000人？,/為什麼/日本/10/月/，/自殺/人數/會破/2000/人/？,\n剛看了一下日本警視廳的資料\n今年1~9月的自殺人數\n都在1400~1800人左右在跑...,\n/剛/看/了/一下/日本/警視廳/的/資料/\n/今年/1/~/9/月/的/自殺/人數/...,20:43:35
1,1,1,0,急！問一首歌，高手請進,/急/！/問/一/首/歌/，/高手/請進,\n宅宅我最近滑臉書\n\n的影片看到一個女的\n\n右手舉起來，甩奶+抖臀\n\n左手舉起...,\n/宅宅/我/最近/滑/臉書/\n/\n/的/影片/看到/一/個/女/的/\n/\n/右手...,16:10:51
2,2,0,0,話說唐吉軻德的消息呢?,/話/說/唐吉軻德/的/消息/呢/?,\n※ 八卦板務請到 GossipPicket 檢舉板實名詢問\n※ a.張貼問卦請注意，充...,\n/※/ /八卦板/務請/到/ /GossipPicket/ /檢舉/板/實名/詢問/\n...,17:21:33
3,3,0,1,網議愛莉莎莎身材如何? 鄉民:根本藏不住,/網議/愛莉/莎莎/身材/如何/?/ /鄉民/:/根本/藏不住,\n各位常常質疑我們莎莎的身材\n\n畢竟平常莎莎穿得比較保守\n\n但昨天在莎莎的影片中\...,\n/各/位/常常/質疑/我們/莎莎/的/身材/\n/\n/畢竟/平常/莎莎/穿得/比較/保...,01:27:23
4,4,1,0,改機車的人怎麼不先去減肥或看醫生啊？,/改/機車/的/人/怎麼/不/先去/減肥/或/看醫生/啊/？,\n每次在路上看到一堆機車蹦蹦叫的\n\n明明就慢到笑，但是吵到死，我認真建議你要不要先去看...,\n/每/次/在/路上/看到/一/堆/機車/蹦蹦/叫/的/\n/\n/明明/就/慢到/笑/，...,10:26:44
...,...,...,...,...,...,...,...,...
31865,31865,1,0,濟州島是那裡比澎湖好玩,/濟州島/是/那裡/比/澎湖/好玩,\n韓國人蜜月會去濟州島\n\n台灣人蜜月不會去澎湖群島\n\n\n一樣是島\n\n澎湖應該...,\n/韓國人/蜜月/會去/濟州島/\n/\n/台灣人/蜜月/不會/去/澎湖群島/\n/\n/...,13:05:25
31866,31866,0,0,中天Youtube是不是更敢講了？,/中天/Youtube/是/不/是/更/敢/講/了/？,\n以前\n中天新聞各種誇張不實\n但還是會稍微裝一下正派\n\n現在\n中天新聞轉Yout...,\n/以前/\n/中天/新聞/各/種/誇張/不實/\n/但/還是/會/稍微/裝/一下/正派/...,22:21:53
31867,31867,0,0,要不要喝水是哪一部電影,/要不要/喝水/是/哪/一部/電影,\n只記得男主角問女主角要不要喝水\n\n這是那部\n\n想複習一下\n\n說也奇怪整部電影...,\n/只/記得/男/主角/問女/主角/要不要/喝水/\n/\n/這是/那/部/\n/\n/想...,16:41:36
31868,31868,0,0,有個和尚一直哭？,/有/個/和尚/一直/哭/？,\n有個和尚眼睛白白的\n\n看起來應該是個瞎子\n\n聽說他的實力是團隊最強的\n\n整天...,\n/有/個/和尚/眼睛/白白/的/\n/\n/看起來/應該/是/個/瞎子/\n/\n/聽說...,23:53:18


In [8]:
my_train = my_train_data[['segment_title' ,'segment_context' ,'push' ,'down']]
my_train

,segment_title,segment_context,push,down
0,/為什麼/日本/10/月/，/自殺/人數/會破/2000/人/？,\n/剛/看/了/一下/日本/警視廳/的/資料/\n/今年/1/~/9/月/的/自殺/人數/...,0,0
1,/急/！/問/一/首/歌/，/高手/請進,\n/宅宅/我/最近/滑/臉書/\n/\n/的/影片/看到/一/個/女/的/\n/\n/右手...,1,0
2,/話/說/唐吉軻德/的/消息/呢/?,\n/※/ /八卦板/務請/到/ /GossipPicket/ /檢舉/板/實名/詢問/\n...,0,0
3,/網議/愛莉/莎莎/身材/如何/?/ /鄉民/:/根本/藏不住,\n/各/位/常常/質疑/我們/莎莎/的/身材/\n/\n/畢竟/平常/莎莎/穿得/比較/保...,0,1
4,/改/機車/的/人/怎麼/不/先去/減肥/或/看醫生/啊/？,\n/每/次/在/路上/看到/一/堆/機車/蹦蹦/叫/的/\n/\n/明明/就/慢到/笑/，...,1,0
...,...,...,...,...
31865,/濟州島/是/那裡/比/澎湖/好玩,\n/韓國人/蜜月/會去/濟州島/\n/\n/台灣人/蜜月/不會/去/澎湖群島/\n/\n/...,1,0
31866,/中天/Youtube/是/不/是/更/敢/講/了/？,\n/以前/\n/中天/新聞/各/種/誇張/不實/\n/但/還是/會/稍微/裝/一下/正派/...,0,0
31867,/要不要/喝水/是/哪/一部/電影,\n/只/記得/男/主角/問女/主角/要不要/喝水/\n/\n/這是/那/部/\n/\n/想...,0,0
31868,/有/個/和尚/一直/哭/？,\n/有/個/和尚/眼睛/白白/的/\n/\n/看起來/應該/是/個/瞎子/\n/\n/聽說...,0,0


In [9]:
df1 = pd.DataFrame(my_train)
df1['segment_title'] = df1['segment_title'].apply(lambda x :  str(x)[1:-2].replace("/"," ").replace("，"," "))
df1['segment_context'] = df1['segment_context'].apply(lambda x :  str(x)[1:-2].replace("/"," ").replace("\n"," ").replace("，"," "))
df1['key'] = df1['segment_title'].astype(str)+' '+df1['segment_context'].astype(str)

In [10]:
# 先將所有的segment_title 切分並放入list中
arr = []
for i in my_train['segment_title']:
    i = str(i)
    arr.append(i.split('/'))

# 將 train的segment_title都放入 dict
wp_point = {} # 算push
wd_point = {} # 算down
for a in arr:
    wp_point.update(dict.fromkeys(a ,1))
    wd_point.update(dict.fromkeys(a ,1))

In [ ]:
wp_point
#G(count(wp_point),wp_point)

In [23]:
df1

,segment_title,segment_context,push,down,key
0,為什麼 日本 10 月 自殺 人數 會破 2000 人,剛 看 了 一下 日本 警視廳 的 資料 今年 1 ~ 9 月 的 自殺 人數 都...,0,0,為什麼 日本 10 月 自殺 人數 會破 2000 人 剛 看 了 一下 日本 警視...
1,急 ！ 問 一 首 歌 高手,宅宅 我 最近 滑 臉書 的 影片 看到 一 個 女 的 右手 舉起來 ...,1,0,急 ！ 問 一 首 歌 高手 宅宅 我 最近 滑 臉書 的 影片 看到 一...
2,話 說 唐吉軻德 的 消息 呢,※ 八卦板 務請 到 GossipPicket 檢舉 板 實名 詢問 ※ ...,0,0,話 說 唐吉軻德 的 消息 呢 ※ 八卦板 務請 到 GossipPicket ...
3,網議 愛莉 莎莎 身材 如何 ? 鄉民 : 根本 藏,各 位 常常 質疑 我們 莎莎 的 身材 畢竟 平常 莎莎 穿得 比較 保守 ...,0,1,網議 愛莉 莎莎 身材 如何 ? 鄉民 : 根本 藏 各 位 常常 質疑 我們 莎莎...
4,改 機車 的 人 怎麼 不 先去 減肥 或 看醫生 啊,每 次 在 路上 看到 一 堆 機車 蹦蹦 叫 的 明明 就 慢到 笑 但是 ...,1,0,改 機車 的 人 怎麼 不 先去 減肥 或 看醫生 啊 每 次 在 路上 看到 一 堆 ...
...,...,...,...,...,...
31865,濟州島 是 那裡 比 澎湖,韓國人 蜜月 會去 濟州島 台灣人 蜜月 不會 去 澎湖群島 一樣 是...,1,0,濟州島 是 那裡 比 澎湖 韓國人 蜜月 會去 濟州島 台灣人 蜜月 不會 去...
31866,中天 Youtube 是 不 是 更 敢 講 了,以前 中天 新聞 各 種 誇張 不實 但 還是 會 稍微 裝 一下 正派 ...,0,0,中天 Youtube 是 不 是 更 敢 講 了 以前 中天 新聞 各 種 誇張 不...
31867,要不要 喝水 是 哪 一部,只 記得 男 主角 問女 主角 要不要 喝水 這是 那 部 想 複習 一下...,0,0,要不要 喝水 是 哪 一部 只 記得 男 主角 問女 主角 要不要 喝水 這是...
31868,有 個 和尚 一直 哭,有 個 和尚 眼睛 白白 的 看起來 應該 是 個 瞎子 聽說 他 的 實...,0,0,有 個 和尚 一直 哭 有 個 和尚 眼睛 白白 的 看起來 應該 是 個 瞎子...


In [12]:
def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() 
    
    return tf

In [75]:
print(tf)

  (0, 91971)	0.22097917077689724
  (0, 90792)	0.1894124247635383
  (0, 84296)	0.3444460691937282
  (0, 78015)	0.07738527864351004
  (0, 71125)	0.08869895297972248
  (0, 69239)	0.06029171834540448
  (0, 68797)	0.1894124247635383
  (0, 60113)	0.18221891238883278
  (0, 58328)	0.23342313754985222
  (0, 46375)	0.09999240346480903
  (0, 44782)	0.10219001956476598
  (0, 38731)	0.10642726125207258
  (0, 32530)	0.09814039505991029
  (0, 24692)	0.12418976109244638
  (0, 23892)	0.06560925179714117
  (0, 23070)	0.16923795074339007
  (0, 22750)	0.3258610319145992
  (0, 18166)	0.06274496571239635
  (0, 16996)	0.09055469771703971
  (0, 14843)	0.1894124247635383
  (0, 14265)	0.1894124247635383
  (0, 13214)	0.1894124247635383
  (0, 12003)	0.1894124247635383
  (0, 11996)	0.18221891238883278
  (0, 9817)	0.15043061875605632
  :	:
  (31868, 60478)	0.1516370342094947
  (31868, 57267)	0.19817476010482651
  (31868, 51622)	0.12291881584955953
  (31868, 44105)	0.23567624165034307
  (31868, 37605)	0.270097715436

In [63]:
df1

,segment_title,segment_context,push,down,key
0,為什麼 日本 10 月 自殺 人數 會破 2000 人,剛 看 了 一下 日本 警視廳 的 資料 今年 1 ~ 9 月 的 自殺 人數 都...,0,0,為什麼 日本 10 月 自殺 人數 會破 2000 人 剛 看 了 一下 日本 警視...
1,急 ！ 問 一 首 歌 高手,宅宅 我 最近 滑 臉書 的 影片 看到 一 個 女 的 右手 舉起來 ...,1,0,急 ！ 問 一 首 歌 高手 宅宅 我 最近 滑 臉書 的 影片 看到 一...
2,話 說 唐吉軻德 的 消息 呢,※ 八卦板 務請 到 GossipPicket 檢舉 板 實名 詢問 ※ ...,0,0,話 說 唐吉軻德 的 消息 呢 ※ 八卦板 務請 到 GossipPicket ...
3,網議 愛莉 莎莎 身材 如何 ? 鄉民 : 根本 藏,各 位 常常 質疑 我們 莎莎 的 身材 畢竟 平常 莎莎 穿得 比較 保守 ...,0,1,網議 愛莉 莎莎 身材 如何 ? 鄉民 : 根本 藏 各 位 常常 質疑 我們 莎莎...
4,改 機車 的 人 怎麼 不 先去 減肥 或 看醫生 啊,每 次 在 路上 看到 一 堆 機車 蹦蹦 叫 的 明明 就 慢到 笑 但是 ...,1,0,改 機車 的 人 怎麼 不 先去 減肥 或 看醫生 啊 每 次 在 路上 看到 一 堆 ...
...,...,...,...,...,...
31865,濟州島 是 那裡 比 澎湖,韓國人 蜜月 會去 濟州島 台灣人 蜜月 不會 去 澎湖群島 一樣 是...,1,0,濟州島 是 那裡 比 澎湖 韓國人 蜜月 會去 濟州島 台灣人 蜜月 不會 去...
31866,中天 Youtube 是 不 是 更 敢 講 了,以前 中天 新聞 各 種 誇張 不實 但 還是 會 稍微 裝 一下 正派 ...,0,0,中天 Youtube 是 不 是 更 敢 講 了 以前 中天 新聞 各 種 誇張 不...
31867,要不要 喝水 是 哪 一部,只 記得 男 主角 問女 主角 要不要 喝水 這是 那 部 想 複習 一下...,0,0,要不要 喝水 是 哪 一部 只 記得 男 主角 問女 主角 要不要 喝水 這是...
31868,有 個 和尚 一直 哭,有 個 和尚 眼睛 白白 的 看起來 應該 是 個 瞎子 聽說 他 的 實...,0,0,有 個 和尚 一直 哭 有 個 和尚 眼睛 白白 的 看起來 應該 是 個 瞎子...


In [110]:
def P1(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['push'].astype(object), test_size=0.07659)
    
    knn = KNeighborsClassifier()
    knn.fit(X_train,y_train.astype('int'))

    y_pred = knn.predict(X_test)
    y_test=y_test.values
    a = [[y_test,y_pred]]
    return y_pred 
def P2(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['down'].astype(object), test_size=0.07659)
    
    knn = KNeighborsClassifier()
    knn.fit(X_train,y_train.astype('int'))

    y_pred = knn.predict(X_test)
    y_test=y_test.values
    a = [[y_test,y_pred]]
    return y_pred 

In [67]:
df['key']

0         為什麼 日本 10 月   自殺 人數 會破 2000 人  剛 看 了 一下 日本 警視...
1         急 ！ 問 一 首 歌   高手   宅宅 我 最近 滑 臉書     的 影片 看到 一...
2         話 說 唐吉軻德 的 消息 呢  ※   八卦板 務請 到   GossipPicket ...
3         網議 愛莉 莎莎 身材 如何 ?   鄉民 : 根本 藏  各 位 常常 質疑 我們 莎莎...
4         改 機車 的 人 怎麼 不 先去 減肥 或 看醫生 啊  每 次 在 路上 看到 一 堆 ...
                               ...                        
31865     濟州島 是 那裡 比 澎湖   韓國人 蜜月 會去 濟州島     台灣人 蜜月 不會 去...
31866     中天 Youtube 是 不 是 更 敢 講 了  以前   中天 新聞 各 種 誇張 不...
31867     要不要 喝水 是 哪 一部   只 記得 男 主角 問女 主角 要不要 喝水     這是...
31868     有 個 和尚 一直 哭  有 個 和尚 眼睛 白白 的     看起來 應該 是 個 瞎子...
31869     消費券 要 怎麼 花 最   噗噗   本肥 現在 還 有 3000 塊 消費券     ...
Name: key, Length: 31870, dtype: object

In [112]:
df1 = my_train
tf = count(df1)
pred_1 = P1(tf,df1)
pred_2 = P2(tf,df1)
dfk_1 = pd.DataFrame(pred_1)
dfk_2 = pd.DataFrame(pred_2)

In [114]:
a = pd.concat([dfk_1,dfk_2],axis=1)
a.to_csv(r'output.csv', index = False)
a

,0,0
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
2436,0,0
2437,0,0
2438,0,0
2439,0,0


var

In [108]:
a = pd.concat([dfk,dfk],axis=1)
a

,0,0
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
2436,0,0
2437,0,0
2438,0,0
2439,0,0


In [28]:
 X_train, X_test, y_train, y_test = train_test_split(tf, df1['push'].astype(object), test_size=0.07659)

In [69]:
print(X_test)

  (0, 102590)	0.1222638787836929
  (0, 99985)	0.10157625088135458
  (0, 98731)	0.1369475987556154
  (0, 96972)	0.08852276373807254
  (0, 96737)	0.0862233329014881
  (0, 95904)	0.1536424232146196
  (0, 91766)	0.13181877439788142
  (0, 90044)	0.12012929287446227
  (0, 78752)	0.19012278976489524
  (0, 72393)	0.18396660311141508
  (0, 57267)	0.11503157534342184
  (0, 47621)	0.10512781660834464
  (0, 47387)	0.3679239022663332
  (0, 47168)	0.134827125668291
  (0, 45479)	0.06532998589966252
  (0, 44525)	0.13390516832942734
  (0, 44421)	0.15532827657578127
  (0, 44180)	0.17357708974965852
  (0, 42744)	0.1672161238197894
  (0, 42704)	0.360578265756275
  (0, 39269)	0.2106606697617329
  (0, 38288)	0.11455660284913328
  (0, 28674)	0.404481377004873
  (0, 28656)	0.11933171219363702
  (0, 23070)	0.3061966885901628
  :	:
  (2439, 20075)	0.118132603365722
  (2439, 20052)	0.09141589260189002
  (2439, 2553)	0.09464173106072069
  (2440, 101976)	0.16813860867312
  (2440, 86839)	0.30779540990241383
  (2440